In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
18207,2024-11-13,Eua Nba,21:30,Philadelphia 76ers,Cleveland Cavaliers,4.52,1.23,216.5,1.84,2.00,8.5,2.02,1.83,6eGLuo76,0.221239,0.813008,0.543478,0.500000,0.034247,261.948,108.070958,0.412566,0.6,1.341641,2.236068,186.12,2.460,0.892777,0.362917,-34.0,164.722,33.933174,0.206003,3.0,0.000000,0.000000,155.89,1.346,0.149933,0.111392,41.0,99,119,1.88,1.31,195.592,192.864,0.809176,0.058926,0.069792,-3.44,-0.688,-5.116279,0.577163,0.3,-0.277163,3.06,0.612,0.375817,0.607820,0.7,0.092180
18208,2024-11-13,Eua Nba,22:00,Houston Rockets,Los Angeles Clippers,1.51,2.67,212.5,1.81,2.04,-5.5,2.00,1.84,jwmYptqn,0.662252,0.374532,0.552486,0.490196,0.036783,229.000,137.973526,0.602504,2.4,1.341641,0.559017,117.70,2.096,1.307375,0.623748,37.0,254.946,113.848940,0.446561,2.4,1.341641,0.559017,417.28,2.232,0.868746,0.389223,26.0,107,128,1.10,3.26,183.414,392.338,0.392461,0.084485,0.058926,1.41,0.282,1.808511,0.671804,0.6,-0.071804,6.56,1.312,1.272866,0.384833,0.6,0.215167
18209,2024-11-13,Eua Nba,22:00,Milwaukee Bucks,Detroit Pistons,1.45,2.87,217.5,1.84,2.00,-5.5,1.93,1.93,ChysqKDb,0.689655,0.348432,0.543478,0.500000,0.038087,228.930,111.184933,0.485672,1.2,1.643168,1.369306,122.76,2.136,1.068050,0.500023,12.0,270.358,56.959399,0.210681,1.8,1.643168,0.912871,284.13,2.392,0.470128,0.196542,12.0,99,123,1.24,2.31,153.106,359.824,0.464857,0.058926,0.000000,-1.14,-0.228,-1.973684,0.606098,0.4,-0.206098,1.26,0.252,7.420635,0.377174,0.4,0.022826
18210,2024-11-13,Eua Nba,22:00,San Antonio Spurs,Washington Wizards,1.23,4.52,221.5,1.81,2.04,-9.5,1.93,1.93,KIvksbrB,0.813008,0.221239,0.552486,0.490196,0.034247,227.316,63.573832,0.279672,1.2,1.643168,1.369306,238.96,2.106,0.711815,0.337994,-4.0,436.692,179.263448,0.410503,0.0,0.000000,NaN,736.00,4.428,2.052016,0.463418,-99.0,116,92,2.06,8.00,211.360,397.498,0.809176,0.084485,0.000000,0.85,0.170,1.352941,0.540754,0.7,0.159246,-1.27,-0.254,-13.858268,0.314427,0.2,-0.114427
18211,2024-11-13,Europa Eurocopa,16:00,Venezia,Turk Telekom,1.83,1.89,155.5,1.81,1.85,-1.5,1.96,1.75,IVPUiQG7,0.546448,0.529101,0.552486,0.540541,0.075549,282.772,313.062646,1.107120,0.6,1.341641,2.236068,144.54,3.622,3.660829,1.010720,-68.0,154.336,51.029347,0.330638,1.2,1.643168,1.369306,154.44,1.944,0.696082,0.358067,-3.0,66,99,2.19,1.56,125.448,205.744,0.022810,0.015456,0.080050,-0.73,-0.146,-5.684932,0.682598,0.7,0.017402,1.43,0.286,3.111888,0.453088,0.4,-0.053088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18350,2024-11-13,Europa Copa Europeia Da Fiba,15:30,CSM Constanta,Cholet,9.35,1.01,167.5,1.86,1.80,19.5,2.01,1.71,U9DUulTO,0.106952,0.990099,0.537634,0.555556,0.097051,482.574,328.408090,0.680534,0.0,0.000000,NaN,271.81,6.836,5.395334,0.789253,-118.0,107.738,11.256657,0.104482,3.0,0.000000,0.000000,93.60,1.164,0.146048,0.125471,88.0,77,90,3.53,1.04,0.000,190.852,1.138469,0.023184,0.114049,0.00,0.000,inf,0.000000,0.0,0.000000,-2.37,-0.474,-0.021097,0.000000,0.0,0.000000
18351,2024-11-13,Europa Copa Europeia Da Fiba,16:00,Bilbao,Prievidza,1.01,16.00,155.5,1.87,1.86,-27.5,2.05,1.60,0AgerSZi,0.990099,0.062500,0.534759,0.537634,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
18213,20:00,Eua Ncaa,Louisiana Tech,UMass,1.82,Back Home
18214,20:30,Eua Ncaa,Bellarmine,Southern Indiana,1.61,Back Home
18239,08:10,Rússia Superliga,Irkutsk,Novosibirsk,1.98,Back Home
18254,19:30,Paraguai Lnb,Colonias Gold,Dep. San Jose,1.83,Back Home
18277,22:00,Eua Ncaa,Western Illinois,Wisc. Green Bay,1.54,Back Home
18290,00:00,Eua Ncaa,California Baptist,UC Riverside,1.63,Back Home
18291,00:00,Eua Ncaa,San Diego Toreros,Portland State,2.14,Back Home
18304,21:00,Eua Nba,Orlando Magic,Indiana Pacers,2.14,Back Home
18307,16:30,Europa Euroliga,Paris,Zalgiris Kaunas,1.80,Back Home
18327,16:30,Itália Série A2,Cento,JuVi Cremona,1.54,Back Home
